<a href="https://colab.research.google.com/github/maberf/colabs/blob/main/Portfolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [186]:
# ##### PORTFOLIO INVESTOR CODE #####
#
# Code to upload data in Google Sheets to support investiments decisions.
# The lists with the tickers and other informations should be filled directly in the code where there are INSERT OR ADJUST HERE lines.
# IFIX historic series csv file with time desired (2y, 3y, etc.) from Investing.com should be downloaded to a Google Drive directory which is in the path in code - https://br.investing.com/indices/bm-fbovespa-real-estate-ifix-historical-data .
# FUNDS EXPLORER - Copy and paste date from https://www.fundsexplorer.com.br/ranking in a Google Drive directory excel file which is in the path in code. It is necessary to SELECT ALL COLUMNS in Funds Explorer website.
# Before to run the code it is necessary to make these activities.  Check the code below until the ENDING POINT TO MANUAL ACTIVITIES.
# Verifiy data on thes files in case of code running problems.

In [187]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import yfinance as yf
import os
import datetime as dt
from google.colab import drive
from google.colab import auth
from google.auth import default
import gspread

In [188]:
# Stock tickers - Insert here# Real Estate Invesments Trust tickers - INSERT OR ADJUST HERE
# Try to mantain IBOV + 25 assets in alphabetical order to easly adjust google docs spreadsheet
tickers = ['^BVSP','ASAI3.SA','AURE3.SA','B3SA3.SA','BBAS3.SA','BBDC4.SA','BBSE3.SA','CGAS3.SA','CPFE3.SA','CSAN3.SA','EGIE3.SA','ELET6.SA','EMBR3.SA','IVVB11.SA','JHSF3.SA','KLBN11.SA','LEVE3.SA','LREN3.SA','ODPV3.SA','PETR4.SA','PRIO3.SA','STBP3.SA','TAEE11.SA','VALE3.SA','VIVT3.SA','WEGE3.SA']

In [189]:
# Real Estate Investments Trust tickers - INSERT OR ADJUST HERE
# Try to mantain IFIX + 20 assets in alphabetical order to easly adjust google docs spreadsheet
# IFIX here only to a space in dataframe, yahoo finance return IFIX.SA maximum 5 days. IFIX should be calculated downloading an Investing.com csv file history and it must be uploaded in personal Google Drive. This code read it.
tickerr = ['IFIX.SA','BTLG11.SA','HGCR11.SA','HGBS11.SA','HGRE11.SA','HGRU11.SA','HSLG11.SA','HSML11.SA','HTMX11.SA','JSAF11.SA','JFLL11.SA','KNCA11.SA','KNHF11.SA','KNIP11.SA','MXRF11.SA','MFII11.SA','SADI11.SA','TGAR11.SA','TRXF11.SA','VGHF11.SA','VISC11.SA']

In [190]:
# US Stocks tickers - INSERT OR ADJUST HERE
# Try to mantain SP500 + USDBRL + 20 assets in alphabetical order to easly adjust google docs spreadsheet
tickersus = ['^GSPC','USDBRL=X','AAPL','AIG','BAC','RIO','DHI','EXC','KMB','KO','LOPE','LYB','MGA','MSFT','MSTR','NUE','NVDA','TGT','TMUS','UPS','UNH','XOM']

In [191]:
# US ETFs tickers - INSERT OR ADJUST HERE
# Try to mantain SP500 + USDBRL + 10 assets in alphabetical order to easly adjust google docs spreadsheet
tickereus = ['^GSPC','USDBRL=X','FBTC','JEPI','HACK', 'IVV','SCHD','SOXX','SPY','TLT','SMH','TFLO']

In [192]:
# Portfolio tickers - INSERT OR ADJUST HERE
tickerport = ['^BVSP','USDBRL=X','BBAS3.SA','BTLG11.SA','CPFE3.SA','ELET6.SA','FBTC','HSML11.SA','HTMX11.SA','IVVB11.SA','PETR4.SA','SADI11.SA','SCHD','TFLO','TGAR11.SA','TRXF11.SA','VALE3.SA']
# Portfolio tickers weight - INSERT OR ADJUST HERE IN THE SAME ORDER!
weightport = [0.0, 0.0, 0.000, 0.000,4.104, 8.341, 4.054, 3.112, 6.119, 9.021, 8.830, 7.681, 31.471, 4.016, 4.437, 5.242, 3.572]
# Portfolio tickers expected returns - INSERT OR ADJUST HERE IN THE SAME ORDER!
# IBOV (^BVSP) MUST always be zero!
expretport = [0.0, 0.0, 0.0, 8.2, 15.5, 5.5, 18.0, 11.3, 26.1, 20.0, 32.1, 20.5, 18.0, 12.0, 28.7, 26.9, 17.0]

In [193]:
# Risk free rate in percentage - INSERT OR ADJUST HERE
riskfree = 15.00

In [194]:
# Target Sharpe in percentage of Sharpe Estimated Maximum - INSERT OR ADJUST HERE
target_per = 75.00

In [195]:
# Portfolio dataframe creation
portfolio = pd.DataFrame({'Ticker': tickerport, 'W': weightport, 'RetE%':expretport})

In [196]:
# Excluding .SA, renaming ^BVSP to IBOV and USDBRL=X to USDBRL
portfolio['Ticker'] = portfolio['Ticker'].str.replace('.SA', '', regex=False)
portfolio['Ticker'] = portfolio['Ticker'].str.replace('^BVSP', 'IBOV', regex=False)
portfolio['Ticker'] = portfolio['Ticker'].str.replace('USDBRL=X', 'USDBRL', regex=False)
# display(portfolio)

In [197]:
# Load tickers history prices in a dataframe considering a certain period of time - ADJUST HERE, default 1 year (1y)
# Sometimes some ticker has problems in yahoo finance. If it happens, close the session and try again. Or change the ticker, because problem in one ticker will cause problem in all code running.
# Check success download completed to all dataframes, otherwise the code will broke in next lines.
dfs = yf.download(tickers, period='2y', auto_adjust=True)['Close']
dfr = yf.download(tickerr, period='2y', auto_adjust=True)['Close']
dfsus = yf.download(tickersus, period='2y', auto_adjust=True)['Close']
dfeus = yf.download(tickereus, period='2y', auto_adjust=True)['Close']
dfport = yf.download(tickerport, period='2y', auto_adjust=True)['Close']
# Remove timezone from index
dfs.index = pd.to_datetime(dfs.index).tz_localize(None)
dfr.index = pd.to_datetime(dfr.index).tz_localize(None)
dfsus.index = pd.to_datetime(dfsus.index).tz_localize(None)
dfeus.index = pd.to_datetime(dfeus.index).tz_localize(None)
dfport.index = pd.to_datetime(dfport.index).tz_localize(None)
# display(dfs)
# display(dfr)
# display(dfeus)
# display(dfport)

[*********************100%***********************]  26 of 26 completed
[*********************100%***********************]  21 of 21 completed
[*********************100%***********************]  22 of 22 completed
[*********************100%***********************]  12 of 12 completed
[*********************100%***********************]  17 of 17 completed


In [198]:
# Convert US assets in BRL values, it should be adjusted according the assets in portfolio - ADJUST HERE
dfport['SCHD'] = dfport['SCHD'] * dfport['USDBRL=X']
dfport['TFLO'] = dfport['TFLO'] * dfport['USDBRL=X']
dfport['FBTC'] = dfport['FBTC'] * dfport['USDBRL=X']
# display(dfport)

In [199]:
# IFIX historic series from Investing.com to be appended in real state dataframe dfr - https://br.investing.com/indices/bm-fbovespa-real-estate-ifix-historical-data - MAKE THIS ACTIVITY
# Download the file from site and copy to your google drive. Rename de file as history.csv. Adjust the path below in " ifixfile = .... " command line according your file location.
# Google Drive mounth
drive.mount('/content/drive', force_remount=True)
# File path on Google Drive - Download the file and upload to Financas folder in Google Drive. Rename the path according file name uploaded.
ifixfile = '/content/drive/MyDrive/Financas/history.csv'
# File csv to dataframe converting quote to float
ifix = pd.read_csv(ifixfile, thousands = '.', decimal = ',', dtype = {'Último':np.float64})
# Excluding and rename columns
ifix = ifix.drop(columns=['Abertura', 'Máxima', 'Mínima', 'Vol.', 'Var%'])
ifix = ifix.rename(columns={'Data': 'Date', 'Último': 'IFIX.SA'})
# Date format in Date column
ifix['Date'] = pd.to_datetime(ifix['Date'], format='%d%m%Y', errors='coerce')
ifix.set_index('Date', inplace=True)
# Solve eventual duplicated registers, grouped by mean
ifix = ifix.groupby(level=0).mean()
# display(ifix)

Mounted at /content/drive


In [200]:
# Go to Funds Explorer Table in https://www.fundsexplorer.com.br/ranking  - MAKE THIS ACTIVITY
# Manually select all columns in web page and paste all data AS VALUES ONLY in a NEW Excel File sheet.
# Save the file with the name fundsexplorer.xlsx in a folder called Financas (or in another preferable path, adjusting the path below)
fundsexplorerfile = '/content/drive/MyDrive/Financas/fundsexplorer.xlsx'
# Excel file reading
fundsexplorer = pd.read_excel(fundsexplorerfile)
#display(fundsexplorer)
# The sequence of FUNDS EXPLORER ROUTINE, see at the end of the code.

In [201]:
# ENDING POINT TO MANUAL ACTIVITIES. The code should run after this point. Permission to Google Drive access can be required during code running

In [202]:
# Replace dfr dataframe by ifix values by index key (date)
dfr.update(ifix)
# display(dfr)

In [203]:
# Excluding .SA, renaming ^BVSP to IBOV, ^GSPC to SP500
dfs.columns = [col.replace('.SA', '') for col in dfs.columns]
dfs.columns = [col.replace('^BVSP', 'IBOV') for col in dfs.columns]
dfr.columns = [col.replace('.SA', '') for col in dfr.columns]
dfsus.columns = [col.replace('^GSPC', 'SP500') for col in dfsus.columns]
dfsus.columns = [col.replace('USDBRL=X', 'USDBRL') for col in dfsus.columns]
dfeus.columns = [col.replace('^GSPC', 'SP500') for col in dfeus.columns]
dfeus.columns = [col.replace('USDBRL=X', 'USDBRL') for col in dfeus.columns]
dfport.columns = [col.replace('.SA', '') for col in dfport.columns]
dfport.columns = [col.replace('^BVSP', 'IBOV') for col in dfport.columns]
dfport.columns = [col.replace('USDBRL=X', 'USDBRL') for col in dfport.columns]

In [204]:
# Exclude NaNs, in the first row and in the dfr Date registers that not included in IFIX excel file.
dfs.dropna(inplace=True)
dfr.dropna(inplace=True)
dfsus.dropna(inplace=True)
dfeus.dropna(inplace=True)
dfport.dropna(inplace=True)
# Other conformations such as ascending order and the market indexes in the first column
dfs = dfs[sorted(dfs.columns)]
dfr = dfr[sorted(dfr.columns)]
dfsus = dfsus[sorted(dfsus.columns)]
dfus = dfeus[sorted(dfeus.columns)]
dfport = dfport[sorted(dfport.columns)]
dfs = dfs[['IBOV'] + [col for col in dfs.columns if col != 'IBOV']]
dfr = dfr[['IFIX'] + [col for col in dfr.columns if col != 'IFIX']]
dfsus = dfsus[['SP500', 'USDBRL'] + [col for col in dfsus.columns if col not in ['SP500', 'USDBRL']]]
dfeus = dfeus[['SP500', 'USDBRL'] + [col for col in dfeus.columns if col not in ['SP500', 'USDBRL']]]
dfport = dfport[['IBOV', 'USDBRL'] + [col for col in dfport.columns if col not in ['IBOV', 'USDBRL']]]
# Here dataframes should be ready for calculations. It will be made and uploaded in dfxvar dataframes later.
# display(dfs)
# display(dfr)
# display(dfsus)
# display(dfeus)
# display(dfport)

In [205]:
# Calculate daily variation
dfsvar = dfs.pct_change()
dfrvar = dfr.pct_change()
dfsusvar = dfsus.pct_change()
dfeusvar = dfeus.pct_change()
dfportvar = dfport.pct_change()
# Excluding MaNs first line
dfsvar.dropna(inplace=True)
dfrvar.dropna(inplace=True)
dfsusvar.dropna(inplace=True)
dfeusvar.dropna(inplace=True)
dfportvar.dropna(inplace=True)
# display(dfsvar)
# display(dfrvar)
# display(dfsusvar)
# display(dfeusvar)
# display(dfportvar)

In [206]:
# Market Percentage Historic Return calculation and column add in output dataframes
stockvar = pd.DataFrame(dfsvar.mean()*252*100).rename(columns={0: 'RetH%'})
stockvar.index.name = 'Ticker'
stockvar['RetH%'] = stockvar['RetH%'].round(1)
realstatevar = pd.DataFrame(dfrvar.mean()*252*100).rename(columns={0: 'RetH%'})
realstatevar.index.name = 'Ticker'
realstatevar['RetH%'] = realstatevar['RetH%'].round(1)
stockusvar = pd.DataFrame(dfsusvar.mean()*252*100).rename(columns={0: 'RetH%'})
stockusvar.index.name = 'Ticker'
stockusvar['RetH%'] = stockusvar['RetH%'].round(1)
etfusvar = pd.DataFrame(dfeusvar.mean()*252*100).rename(columns={0: 'RetH%'})
etfusvar.index.name = 'Ticker'
etfusvar['RetH%'] = etfusvar['RetH%'].round(1)
portvar = pd.DataFrame(dfportvar.mean()*252*100).rename(columns={0: 'RetH%'})
portvar.index.name = 'Ticker'
portvar['RetH%'] = portvar['RetH%'].round(1)
# display(stockvar)
# display(realstatevar)
# display(stockusvar)
# display(etfusvar)
# display(portvar)

In [207]:
# Market return variance calculation
vars = dfsvar.var()*252
varr = dfrvar.var()*252
varsus = dfsusvar.var()*252
vareus = dfeusvar.var()*252
varport = dfportvar.var()*252
# display(vars)
# display(varr)
# display(varsus)
# display(vareus)
# display(varport)

In [208]:
# Market risk calculation, in percentage (%). Add column in output dataframes
stockvar['Risk%'] = dfsvar.std()*np.sqrt(252)*100
stockvar['Risk%'] = stockvar['Risk%'].round(0)
realstatevar['Risk%'] = dfrvar.std()*np.sqrt(252)*100
realstatevar['Risk%'] = realstatevar['Risk%'].round(0)
stockusvar['Risk%'] = dfsusvar.std()*np.sqrt(252)*100
stockusvar['Risk%'] = stockusvar['Risk%'].round(0)
etfusvar['Risk%'] = dfeusvar.std()*np.sqrt(252)*100
etfusvar['Risk%'] = etfusvar['Risk%'].round(0)
portvar['Risk%'] = dfportvar.std()*np.sqrt(252)*100
portvar['Risk%'] = portvar['Risk%'].round(0)
# display(stockvar)
# display(realstatevar)
# display(stockusvar)
# display(etfusvar)
# display(portvar)

In [209]:
# Covariance calculation
covs = dfsvar.cov()*252
covr = dfrvar.cov()*252
covsus = dfsusvar.cov()*252
coveus = dfeusvar.cov()*252
covport = dfportvar.cov()*252
# display(covs)
# display(covr)
# display(covsus)
# display(coveus)
# display(covport)

In [210]:
# Beta calculation
betas = covs['IBOV']/vars['IBOV']
betas = betas.round(3)
betas.name = 'Beta'
betar = covr['IFIX']/varr['IFIX']
betar = betar.round(3)
betar.name = 'Beta'
betasus = covsus['SP500']/varsus['SP500']
betasus = betasus.round(3)
betasus.name = 'Beta'
betaeus = coveus['SP500']/vareus['SP500']
betaeus = betaeus.round(3)
betaeus.name = 'Beta'
betaport = covport['IBOV']/varport['IBOV']
betaport = betaport.round(3)
betaport.name = 'Beta'
# display(betas)
# display(betar)
# display(betasus)
# display(betaeus)
# display(betaport)

In [211]:
# Adding Beta to column output dataframes
stockvar['Beta'] = betas
realstatevar['Beta'] = betar
stockusvar['Beta'] = betasus
etfusvar['Beta'] = betaeus
portvar['Beta'] = betaport
# display(stockvar)
# display(realstatevar)
# display(stockusvar)
# display(etfusvar)
# display(portvar)

In [212]:
# Adding Min to column output dataframes
stockvar['Min'] = dfs.min()
stockvar['Min'] = stockvar['Min'].round(2)
realstatevar['Min'] = dfr.min()
realstatevar['Min'] = realstatevar['Min'].round(2)
stockusvar['Min'] = dfsus.min()
stockusvar['Min'] = stockusvar['Min'].round(2)
etfusvar['Min'] = dfeus.min()
etfusvar['Min'] = etfusvar['Min'].round(2)
portvar['Min'] = dfport.min()
portvar['Min'] = portvar['Min'].round(2)
# display(stockvar)
# display(realstatevar)
# display(stockusvar)
# display(etfusvar)
# display(portvar)

In [213]:
# Adding Max to column output dataframes
stockvar['Max'] = dfs.max()
stockvar['Max'] = stockvar['Max'].round(2)
realstatevar['Max'] = dfr.max()
realstatevar['Max'] = realstatevar['Max'].round(2)
stockusvar['Max'] = dfsus.max()
stockusvar['Max'] = stockusvar['Max'].round(2)
etfusvar['Max'] = dfeus.max()
etfusvar['Max'] = etfusvar['Max'].round(2)
portvar['Max'] = dfport.max()
portvar['Max'] = portvar['Max'].round(2)
# display(stockvar)
# display(realstatevar)
# display(stockusvar)
# display(etfusvar)
# display(portvar)

In [214]:
# Organizing columns order and making Index column as index of dataframes
stockvar = stockvar.reset_index()
realstatevar = realstatevar.reset_index()
stockusvar = stockusvar.reset_index()
etfusvar = etfusvar.reset_index()
portvar = portvar.reset_index()
# display(stockvar)
# display(realstatevar)
# display(stockusvar)
# display(etfusvar)
# display(portvar)

In [215]:
# FUNCTIONS TO CALCULATE PORTFOLIO ERFORMANCE, MAXIMUM SHARPE AND TARGET SHARPE
# --- Exemplo de uso ---
# Supondo que você tem df_returns (DataFrame) com retornos log ou simples, calculados diariamente/mensalmente:
# mu = df_returns.mean().values           # retorno médio por ativo (periodicidade compatível)
# cov = df_returns.cov().values           # matriz de covariância
# risk_free = 0.0                         # ajuste conforme sua periodicidade (ex: taxa anual/252 para dia)
#
# res_max, w_max = max_sharpe_weights(mu, cov, riskfree)
# print("Max Sharpe:", portfolio_performance(w_max, mu, cov, risk_free)[2], "pesos:", w_max)
#
# target = 1.2
# res_tgt, w_tgt, info = weights_for_target_sharpe(mu, cov, target, riskfree)
# print("Resultado objetivo:", info)
# print("Pesos:", w_tgt)

In [216]:
# Portfolio dataframe assembling
portfolio = pd.merge(portfolio, portvar, on='Ticker', how='inner')
# portfolio = portfolio[sorted(portfolio.columns)]
# portfolio = portfolio[['IBOV', 'USDBRL'] + [col for col in portfolio.columns if col not in ['IBOV', 'USDBRL']]]
# firstsnames = ['IBOV','USDBRL']
firstsnames = ['IBOV', 'USDBRL']
portfolio.iloc[2:] = portfolio[~portfolio['Ticker'].isin(firstsnames)].sort_values(by='Ticker').values
# display(portfolio)

In [217]:
# Portfolio PERFORMANCE calculation function
def portfolio_performance(weights, mu, cov, riskfree):
    """
    Retorna (retorno_portfolio, volatilidade_portfolio, sharpe_portfolio)
    expected_returns: vetor de retornos esperados por ativo (numpy array)
    cov: matriz de covariância (numpy array)
    weights: vetor de pesos
    riskfree: taxa livre de risco (mesma periodicidade)
    """
    w = np.array(weights)
    ret = float(np.dot(w, mu))
    vol = float(np.sqrt(w.T @ cov @ w))
    if vol == 0:
        sharpe = 0.0
    else:
        sharpe = (ret - riskfree) / vol
    return ret, vol, sharpe

In [218]:
riskfree = riskfree / 100 # Risk Free is indicated in percentage

In [219]:
weights = [values / 100 for values in weightport] # Expected Returns are indicated in percentage

In [220]:
# Convert covariance dataframe in numpy matrix
cov = covport.values

In [221]:
# Extracting expected returns
mu = portfolio['RetE%'].values / 100

In [222]:
# Porfolio Expected Return, Expected Risk and Expected Sharpe calculations
portfolioretexptotal, portfolioriskexptotal, portfoliosharpeexp = portfolio_performance(weights, mu, cov, riskfree)
# print (portfolioretexptotal, portfolioriskexptotal, portfoliosharpeexp)

In [223]:
# Portfolio adding columns with Total Return, Total Risk and Sharpe values in first register(in the same line of IBOV value index). Other registers being filled with zero.
portfolio['RetET%'] = [portfolioretexptotal * 100] + [0] * (len(portfolio) - 1)
portfolio['RetET%'] = portfolio['RetET%'].round(1)
portfolio['RiskET%'] = [portfolioriskexptotal * 100] + [0] * (len(portfolio) - 1)
portfolio['RiskET%'] = portfolio['RiskET%'].round(0)
portfolio['SharpeE'] = [portfoliosharpeexp] + [0] * (len(portfolio) - 1)
portfolio['SharpeE'] = portfolio['SharpeE'].round(3)

In [224]:
# Extracting real returns
mu = portfolio['RetH%'].values / 100

In [225]:
# Porfolio Real (historic) Return, Risk and Sharpe calculations
portfoliorettotal, portfoliorisktotal, portfoliosharpe = portfolio_performance(weights, mu, cov, riskfree)
# print (portfoliorettotal, portfoliorisktotal, portfoliosharpe)

In [226]:
portfolio['RetHT%'] = [portfoliorettotal * 100] + [0] * (len(portfolio) - 1)
portfolio['RetHT%'] = portfolio['RetHT%'].round(1)
portfolio['SharpeH'] = [portfoliosharpe] + [0] * (len(portfolio) - 1)
portfolio['SharpeH'] = portfolio['SharpeH'].round(3)

In [227]:
# Calculate Total Beta and add in IBOV line
portfolio.at[0, 'Beta'] = (portfolio['W'] / 100 * portfolio['Beta']).sum()
portfolio['Beta'] = portfolio['Beta'].round(3)
# display(portfolio)

In [228]:
# Portfolio MAXIMUM SHARPE calculation function
def max_sharpe_weights(mu, cov, risk_free, bounds=None, constraints=None):
    """
    Encontra os pesos que maximizam o Sharpe (usando minimize sobre -sharpe).
    bounds: lista de tuplas (min, max) por ativo. padrão: (0,1) para todos.
    constraints: lista de constraint dicts para scipy.optimize.minimize (opcional).
    Retorna o dicionário de resultado e os pesos ótimos.
    """
    n = len(mu)
    if bounds is None:
        bounds = tuple((0.0, 1.0) for _ in range(n))
    # restrição: soma dos pesos = 1
    cons = [{'type':'eq', 'fun': lambda w: np.sum(w) - 1.0}]
    if constraints:
        cons.extend(constraints)
    # inicialização: pesos iguais
    x0 = np.ones(n) / n

    def neg_sharpe(w):
        _, _, s = portfolio_performance(w, mu, cov, riskfree)
        return -s

    res = minimize(neg_sharpe, x0, method='SLSQP', bounds=bounds, constraints=cons)
    return res, res.x if res.success else None

In [229]:
# Extracting expected returns to calculte Sharpe Maximum
mu = portfolio['RetE%'].values / 100

In [230]:
res_max, w_max = max_sharpe_weights(mu, cov, riskfree)
print("Max Sharpe:", portfolio_performance(w_max, mu, cov, riskfree)[2], "pesos:", w_max)
# print("Ret Max Sharpe:", w_max)

Max Sharpe: 1.4969204388169215 pesos: [9.95724372e-16 0.00000000e+00 0.00000000e+00 3.12215424e-16
 0.00000000e+00 6.81310643e-16 0.00000000e+00 3.28754312e-16
 1.03478640e-01 3.51760172e-02 1.38164490e-01 1.41049659e-01
 0.00000000e+00 0.00000000e+00 5.15265067e-02 5.30604688e-01
 6.54807752e-17]


In [231]:
#  Add Maximum Sharpe ticker weights to portfolio dataframe
portfolio['ShMaxE-W'] = w_max.round(3) * 100
# display(portfolio)

In [232]:
# Add Maximum Sharpe value to portfolio dataframe in the first line
portfolio.at[0, 'ShMaxE-W'] = round(portfolio_performance(w_max, mu, cov, riskfree)[2], 3)
# display(portfolio)

In [233]:
# Add Sharpe Maximum Return column and the value in the first line
portfolio['ShMaxRetE%'] = [portfolio_performance(w_max, mu, cov, riskfree)[0] *100] + [0] * (len(portfolio) - 1)
portfolio['ShMaxRetE%'] = portfolio['ShMaxRetE%'].round(1)
# display(portfolio)

In [234]:
# Portfolio TARGET SHARPE calculation function
def weights_for_target_sharpe(mu, cov, target_sharpe, riskfree, bounds=None, tol=1e-6, maxiter=1000):
    """
    Encontra pesos que aproximem um Sharpe alvo, minimizando o erro quadrático entre Sharpe(weights) e target_sharpe.
    Retorna o resultado da otimização e os pesos.
    Observação: A solução pode ser inviável exatamente; a rotina busca o vetor que minimize o erro.
    """
    n = len(mu)
    if bounds is None:
        bounds = tuple((0.0, 1.0) for _ in range(n))

    # restrição: soma dos pesos = 1
    cons = [{'type':'eq', 'fun': lambda w: np.sum(w) - 1.0}]

    # inicialização: use o max-sharpe como chute inicial, se possível
    try:
        res_max, w0 = max_sharpe_weights(mu, cov, riskfree, bounds=bounds)
        if w0 is None:
            x0 = np.ones(n) / n
        else:
            x0 = w0
    except Exception:
        x0 = np.ones(n) / n

    def sharpe_err_sq(w):
        _, _, s = portfolio_performance(w, mu, cov, riskfree)
        return (s - target_sharpe)**2

    opts = {'maxiter': maxiter, 'ftol': tol}
    res = minimize(sharpe_err_sq, x0, method='SLSQP', bounds=bounds, constraints=cons, options=opts)

    # informação extra: distância do objetivo e performance
    if res.success:
        ret, vol, s = portfolio_performance(res.x, mu, cov, riskfree)
        info = {'target_sharpe': target_sharpe, 'achieved_sharpe': s, 'ret': ret, 'vol': vol}
    else:
        info = {'message': res.message}

    return res, res.x if res.success else None, info


In [235]:
# TARGET SHARPE definition
# Target the average between Estimated Sharpe and Maximum Estimated Sharpe
# target = portfolio.at[0, 'SharpeE'] + ((portfolio.at[0, 'ShMaxE-W'] - portfolio.at[0, 'SharpeE']) / 2)
# Target as percentage of Maximum Estimated Sharpe
target = portfolio.at[0, 'ShMaxE-W'] * target_per / 100
print("Target:", target)
res_tgt, w_tgt, info = weights_for_target_sharpe(mu, cov, target, riskfree)
# print("Resultado objetivo:", info)
# print("Pesos:", w_tgt)
# print("Achieved Sharpe:", info['achieved_sharpe'])

Target: 1.1227500000000001


In [236]:
#  Add Target Sharpe ticker weights to portfolio dataframe
portfolio['ShTg-W'] = (w_tgt * 100).round(3)
# Add Target (Achieved) Sharpe value to portfolio dataframe in the first line
portfolio.at[0, 'ShTg-W'] = round(info['achieved_sharpe'], 3)
# Add Target Maximum Return column and the value in the first line
portfolio['ShTgRetE%'] = [portfolio_performance(w_tgt, mu, cov, riskfree)[0] *100] + [0] * (len(portfolio) - 1)
portfolio['ShTgRetE%'] = portfolio['ShTgRetE%'].round(1)
display(portfolio)

,Ticker,W,RetE%,RetH%,Risk%,Beta,Min,Max,RetET%,RiskET%,SharpeE,RetHT%,SharpeH,ShMaxE-W,ShMaxRetE%,ShTg-W,ShTgRetE%
0,IBOV,0.000,0.0,6.4,14.0,0.357,118533.00,146492.00,19.4,10.0,0.427,14.2,-0.078,1.497,26.5,1.123,23.5
1,USDBRL,0.000,0.0,6.7,13.0,-0.007,4.86,6.30,0.0,0.0,0.000,0.0,0.000,0.000,0.0,0.036,0.0
2,BBAS3,0.000,0.0,-5.9,23.0,0.838,18.35,29.22,0.0,0.0,0.000,0.0,0.000,0.000,0.0,0.000,0.0
3,BTLG11,0.000,8.2,11.2,12.0,-0.014,79.11,103.47,0.0,0.0,0.000,0.0,0.000,0.000,0.0,6.511,0.0
4,CPFE3,4.104,15.5,4.0,20.0,0.637,31.00,41.23,0.0,0.0,0.000,0.0,0.000,0.000,0.0,0.012,0.0
5,ELET6,8.341,5.5,20.5,22.0,1.023,35.38,57.34,0.0,0.0,0.000,0.0,0.000,0.000,0.0,0.001,0.0
6,FBTC,4.054,18.0,68.0,53.0,0.527,171.30,583.82,0.0,0.0,0.000,0.0,0.000,0.000,0.0,0.000,0.0
7,HSML11,3.112,11.3,4.1,15.0,0.142,64.45,85.98,0.0,0.0,0.000,0.0,0.000,0.000,0.0,5.364,0.0
8,HTMX11,6.119,26.1,3.9,24.0,-0.104,121.33,179.05,0.0,0.0,0.000,0.0,0.000,10.300,0.0,11.099,0.0
9,IVVB11,9.021,20.0,27.7,17.0,-0.078,257.48,417.20,0.0,0.0,0.000,0.0,0.000,3.500,0.0,0.007,0.0


In [237]:
# Autentication in Google Docs (only once)
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [238]:
# Open workbook and worksheets
wb = gc.open('Quotes')
wss = wb.worksheet('Stockvar')
wsr = wb.worksheet('RealStatevar')
wssus = wb.worksheet('StockUSvar')
wseus = wb.worksheet('ETFUSvar')
wsport = wb.worksheet('Portfolio')

In [239]:
# Write data in the worksheets
wss.update([stockvar.columns.values.tolist()] + stockvar.values.tolist())
wsr.update([realstatevar.columns.values.tolist()] + realstatevar.values.tolist())
wssus.update([stockusvar.columns.values.tolist()] + stockusvar.values.tolist())
wseus.update([etfusvar.columns.values.tolist()] + etfusvar.values.tolist())
wsport.update([portfolio.columns.values.tolist()] + portfolio.values.tolist())

{'spreadsheetId': '1qgTSxri55kYWVahW6sH3Fbn3ofWzhq93umUJhcwO7Uk',
 'updatedRange': 'Portfolio!A1:Q18',
 'updatedRows': 18,
 'updatedColumns': 17,
 'updatedCells': 306}

In [240]:
# FUNDS EXPLORER ROUTINE
#
# tickerr = ['IFIX.SA','BTLG11.SA','HGCR11.SA','HGBS11.SA','HGRE11.SA','HGRU11.SA','HSLG11.SA','HSML11.SA','HTMX11.SA','JSAF11.SA','JFLL11.SA','KNCA11.SA','KNHF11.SA','KNIP11.SA','MALL11.SA','MFII11.SA','SADI11.SA','TGAR11.SA','TRXF11.SA','VGHF11.SA','VISC11.SA']
# Remove ".SA" from real state funds dataframe tickers
tickerradjusted = [nome.replace('.SA', '') for nome in tickerr]
# Remove "IFIX" ticker
if "IFIX" in tickerradjusted:
    tickerradjusted.remove("IFIX")
#

In [241]:
# Funds list filter, exclude tickers not in real state funds list
fundsexplorer = fundsexplorer[fundsexplorer['Fundos'].isin(tickerradjusted)]
# display(fundsexplorer)

In [242]:
# Open worksheet
wsrfunds = wb.worksheet('FundsExplorer')

In [243]:
# Write data in the worksheets
wsrfunds.update([fundsexplorer.columns.values.tolist()] + fundsexplorer.fillna('').values.tolist())

{'spreadsheetId': '1qgTSxri55kYWVahW6sH3Fbn3ofWzhq93umUJhcwO7Uk',
 'updatedRange': 'FundsExplorer!A1:AD21',
 'updatedRows': 21,
 'updatedColumns': 30,
 'updatedCells': 630}